In [192]:
import pandas as pd

from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score

from matplotlib import pyplot as plt

In [160]:
df = pd.read_excel("./calls.xlsx")
df.drop(['INDEX()'],axis=1,inplace=True)
df['Call Date'] = pd.to_datetime(df['Call Date'], format="%d.%m.%Y")
df['Week of Call Date'] = df['Week of Call Date'].str.replace('Week ','').astype(int)
#df = pd.get_dummies(df, columns=['Call Reason','Customer'])

df = df[['Call Date','Month','Year of Call Date','Week of Call Date','Day of Call Date','Weekday of Call Date','Total Incoming Calls']]
#df = df.groupby(['Call Date','Year of Call Date','Month','Week of Call Date','Day of Call Date','Weekday of Call Date']).sum('Total Incoming Calls')
#df.reset_index(inplace=True)
#df['Day'] = pd.to_datetime(df['Call Date']).dt.day
#df['WeekDay'] = pd.to_datetime(df['Call Date']).dt.dayofweek
df

,Call Date,Month,Year of Call Date,Week of Call Date,Day of Call Date,Weekday of Call Date,Total Incoming Calls
0,2020-01-17,1,2020,3,17,6,296
1,2020-01-20,1,2020,4,20,2,381
2,2020-01-21,1,2020,4,21,3,363
3,2020-01-22,1,2020,4,22,4,305
4,2020-01-23,1,2020,4,23,5,304
...,...,...,...,...,...,...,...
482,2021-12-09,12,2021,50,9,5,882
483,2021-12-10,12,2021,50,10,6,792
484,2021-12-13,12,2021,51,13,2,941
485,2021-12-14,12,2021,51,14,3,948


In [161]:
df.dtypes

Call Date               datetime64[ns]
Month                            int64
Year of Call Date                int64
Week of Call Date                int64
Day of Call Date                 int64
Weekday of Call Date             int64
Total Incoming Calls             int64
dtype: object

In [162]:
#Breaking the data and selecting features , predictors
from sklearn.model_selection import train_test_split
predictors=df.drop(['Total Incoming Calls','Call Date'],axis=1)
target=df['Total Incoming Calls']
x_train,x_cv,y_train,y_cv=train_test_split(predictors,target,test_size=0.2,random_state=42)

#Comparing Algorithms
def scores(i):
    lin = i()
    lin.fit(x_train, y_train)
    y_pred=lin.predict(x_cv)
    lin_r= r2_score(y_cv, y_pred)
    s.append(lin_r)
#Checking the scores by using our function
algos=[LinearRegression,KNeighborsRegressor,
       RandomForestRegressor,Lasso,ElasticNet,DecisionTreeRegressor]
s=[]
for i in algos:
    scores(i)

models = pd.DataFrame({
    'Method': ['LinearRegression', 'KNeighborsRegressor',
               'RandomForestRegressor', 'Lasso','DecisionTreeRegressor'],
    'Score': [s[0],s[1],s[2],s[3],s[4]]})
models.sort_values(by='Score', ascending=False)

,Method,Score
2,RandomForestRegressor,0.852948
1,KNeighborsRegressor,0.728191
0,LinearRegression,0.540051
3,Lasso,0.453211
4,DecisionTreeRegressor,0.435330


In [182]:
#Hypertuned Model
model = RandomForestRegressor()
model.fit(x_train,y_train)
pred=model.predict(x_cv)
r2_score(pred,y_cv)

0.8248711866721407

In [208]:
test = pd.read_excel('./calls_test.xlsx')
test['Call Date'] = pd.to_datetime(test['Call Date'], format="%d.%m.%Y")
test1=test.drop(['Call Date'],axis=1)
pred2=model.predict(test1)
test['Total Incoming Calls']=pred2

test

FileNotFoundError: [Errno 2] No such file or directory: 'calls_test.xlsx'

In [205]:
model.feature_importances_

array([0.03032633, 0.01723354, 0.82892321, 0.06473302, 0.05878389])